In [1]:
import numpy as np

# Метод трапеций
def integrate_trapezoid(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    integral = h * (0.5 * f(x[0]) + 0.5 * f(x[-1]) + np.sum(f(x[1:-1])))
    return integral

# Метод прямоугольников
def integrate_rectangle(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b - h, n) + h / 2
    integral = h * np.sum(f(x))
    return integral

# Функция для построения кубического сплайна
def cubic_spline(x, y):
    n = len(x) - 1
    h = np.diff(x)
    alpha = np.zeros(n)
    l = np.ones(n + 1)
    mu = np.zeros(n + 1)
    z = np.zeros(n + 1)

    for i in range(1, n):
        alpha[i] = (3 / h[i] * (y[i + 1] - y[i]) - 3 / h[i - 1] * (y[i] - y[i - 1]))

    for i in range(1, n):
        l[i] = 2 * (x[i + 1] - x[i - 1]) - h[i - 1] * mu[i - 1]
        mu[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i - 1] * z[i - 1]) / l[i]

    c = np.zeros(n + 1)
    b = np.zeros(n)
    d = np.zeros(n)
    a = y[:-1]

    for j in range(n - 1, -1, -1):
        c[j] = z[j] - mu[j] * c[j + 1]
        b[j] = (y[j + 1] - y[j]) / h[j] - h[j] * (c[j + 1] + 2 * c[j]) / 3
        d[j] = (c[j + 1] - c[j]) / (3 * h[j])

    return a, b, c, d, x

# Интегрирование кубическим сплайном
def spline_integral(a, b, c, d, x):
    integral = 0
    for i in range(len(a)):
        h = x[i + 1] - x[i]
        integral += (a[i] * h + b[i] * h**2 / 2 + c[i] * h**3 / 3 + d[i] * h**4 / 4)
    return integral

# Исходная функция
f_pi = lambda x: 4 / (1 + x**2)

# Значения n
n_values = [8, 32, 128]

# Вывод результатов
print("Формулы трапеции и прямоугольника для π:")
print("n     Трапеции    Error   Прямоугольник    Error")
for n in n_values:
    trapezoid_result = integrate_trapezoid(f_pi, 0, 1, n)
    rectangle_result = integrate_rectangle(f_pi, 0, 1, n)
    print(f"{n:3d}   {trapezoid_result:.8f}   {abs(np.pi - trapezoid_result):.2e}   "
          f"{rectangle_result:.8f}   {abs(np.pi - rectangle_result):.2e}")

print("\nСплайн квадратуры для π:")
print("n     Сплайн           Error")
for n in n_values:
    x = np.linspace(0, 1, n + 1)
    y = f_pi(x)
    a, b, c, d, x = cubic_spline(x, y)
    spline_result = spline_integral(a, b, c, d, x)
    print(f"{n:3d}   {spline_result:.8f}   {abs(np.pi - spline_result):.2e}")


Формулы трапеции и прямоугольника для π:
n     Трапеции    Error   Прямоугольник    Error
  8   3.13898849   2.60e-03   3.14289473   1.30e-03
 32   3.14142989   1.63e-04   3.14167403   8.14e-05
128   3.14158248   1.02e-05   3.14159774   5.09e-06

Сплайн квадратуры для π:
n     Сплайн           Error
  8   3.14130565   2.87e-04
 32   3.14158824   4.41e-06
128   3.14159258   6.88e-08


In [2]:
import numpy as np
from scipy.special import erf

# Функция ошибки для численного интегрирования
def erf_numeric(x, n=1000):
    a, b = 0, x  # пределы интегрирования
    h = (b - a) / n  # шаг
    t = np.linspace(a, b, n + 1)
    y = (2 / np.sqrt(np.pi)) * np.exp(-t**2)
    integral = h * (np.sum(y[:-1]) + np.sum(y[1:])) / 2
    return integral

# Генерация таблицы значений
x_values = np.arange(0.0, 2.1, 0.1)
erf_values_numeric = [erf_numeric(x) for x in x_values]
erf_values_exact = [erf(x) for x in x_values]

# Вывод таблицы
print(" x    erf через интеграл   erf табличное     Error")
for x, num, exact in zip(x_values, erf_values_numeric, erf_values_exact):
    print(f"{x:4.1f}   {num}   {exact}   {abs(num - exact):.2e}")

 x    erf через интеграл   erf табличное     Error
 0.0   0.0   0.0   0.00e+00
 0.1   0.11246291583209296   0.1124629160182849   1.86e-10
 0.2   0.2227025877649654   0.22270258921047845   1.45e-09
 0.3   0.3286267548184533   0.32862675945912745   4.64e-09
 0.4   0.4283923447902278   0.42839235504666845   1.03e-08
 0.5   0.5204998595050758   0.5204998778130465   1.83e-08
 0.6   0.6038560625071617   0.6038560908479259   2.83e-08
 0.7   0.6778011543195378   0.6778011938374184   3.95e-08
 0.8   0.7421009139355382   0.7421009647076605   5.08e-08
 0.9   0.796908151433648   0.7969082124228322   6.10e-08
 1.0   0.8427007237651296   0.8427007929497148   6.92e-08
 1.1   0.8802049949316891   0.8802050695740817   7.46e-08
 1.2   0.910313901234463   0.9103139782296354   7.70e-08
 1.3   0.9340078687018297   0.9340079449406524   7.62e-08
 1.4   0.9522850470733122   0.9522851197626488   7.27e-08
 1.5   0.9661050795770306   0.9661051464753108   6.69e-08
 1.6   0.9763483237962841   0.976348383344644   5

In [3]:
import numpy as np

#Для этого мы можем использовать метод Симпсона
# (он подходит для гладких функций и обеспечивает высокую точность
# с меньшим числом разбиений), метод трапеций или метод прямоугольников.

def f(x):
    if 0 <= x <= 2:
        return np.exp(x ** 2)  # e^(x^2) для 0 <= x <= 2
    elif 2 < x <= 4:
        return 1 / (4 - np.sin(16 * np.pi * x))  # 1 / (4 - sin(16 * pi * x)) для 2 <= x <= 4
    else:
        return 0


# Метод Симпсона для численного интегрирования
def simpsons_method(a, b, n):
    h = (b - a) / n  # шаг
    integral = f(a) + f(b)

    for i in range(1, n, 2):
        integral += 4 * f(a + i * h)
    for i in range(2, n - 1, 2):
        integral += 2 * f(a + i * h)

    integral *= h / 3
    return integral

a1, b1 = 0, 2
a2, b2 = 2, 4
n = 100

# Интеграл на интервале [0, 2]
integral1 = simpsons_method(a1, b1, n)

# Интеграл на интервале [2, 4] с методом Симпсона
integral2 = simpsons_method(a2, b2, n)

total_integral = integral1 + integral2

print(f"Интеграл на интервале [0, 2]: {integral1}")
print(f"Интеграл на интервале [2, 4] методом Симпсона: {integral2}")
print(f"Общий интеграл: {total_integral}")

Интеграл на интервале [0, 2]: 16.452632030425274
Интеграл на интервале [2, 4] методом Симпсона: 0.8787187797152844
Общий интеграл: 17.33135081014056
